# Автоматический выбор числа компонент
---
Часто заранее неизвестно, сколько главных компонент следует оставить. Один из подходов – задать порог по доле объяснённой дисперсии (например, 95% или 99%) и выбрать минимальное $k$, при котором этот порог достигается. Используя уже вычисленные собственные значения, можно последовательно суммировать их до тех пор, пока сумма не достигнет $threshold \times \sum_{i=1}^m \lambda_i$.

---

In [3]:
from src.auto_select_k import auto_select_k
import pytest

---
## Стандартные сценарии выбора `k`


In [8]:
tests = [
    # простой убывающий список: порог 0.5 -> минимум 2 компоненты из [5,4,3,2,1] дают (5+4)/(5+4+3+2+1)=9/15=0.6 ≥ 0.5
    ([5, 4, 3, 2, 1], 0.5, 2),
    # порог 0.9 -> нужно k=3: (5+4+3)/15 = 12/15 = 0.8 <0.9, k=4: 14/15 ≈0.933 ≥0.9
    ([5, 4, 3, 2, 1], 0.9, 4),
    # равные собственные значения: при любых k порог 0.5 для [1,1,1,1] -> k=2: 2/4 =0.5
    ([1, 1, 1, 1], 0.5, 2),
    # один элемент
    ([10], 0.95, 1),
    # пустой список
    ([], 0.75, 0),
]

for evs, thr, expected in tests:
    k = auto_select_k(evs, thr)
    print(f"eigenvalues={evs}, threshold={thr} → k={k} (expected {expected})")
    assert k == expected, f"Expected {expected}, got {k}"

eigenvalues=[5, 4, 3, 2, 1], threshold=0.5 → k=2 (expected 2)
eigenvalues=[5, 4, 3, 2, 1], threshold=0.9 → k=4 (expected 4)
eigenvalues=[1, 1, 1, 1], threshold=0.5 → k=2 (expected 2)
eigenvalues=[10], threshold=0.95 → k=1 (expected 1)
eigenvalues=[], threshold=0.75 → k=0 (expected 0)


- **`[5, 4, 3, 2, 1], threshold=0.5 → k=2`**
  Сумма собственных значений = 15.
  - 1 компонента: 5/15 ≈ 0.33 < 0.5
  - 2 компоненты: (5+4)/15 = 9/15 = 0.60 ≥ 0.5
  **Вывод:** минимальное `k=2`.

- **`[5, 4, 3, 2, 1], threshold=0.9 → k=4`**
  - (5+4+3)/15 = 12/15 = 0.80 < 0.9
  - (5+4+3+2)/15 = 14/15 ≈ 0.933 ≥ 0.9
  **Вывод:** минимальное `k=4`.

- **`[1, 1, 1, 1], threshold=0.5 → k=2`**
  Все λ одинаковы, суммарная дисперсия = 4.
  - 1 компонента: 1/4 = 0.25 < 0.5
  - 2 компоненты: 2/4 = 0.50 ≥ 0.5
  **Вывод:** минимальное `k=2`.

- **`[10], threshold=0.95 → k=1`**
  Одна компонента, её вклад = 100% ≥ 95%.
  **Вывод:** `k=1`.

- **`[], threshold=0.75 → k=0`**
  Нет собственных значений → возвращается 0.
  **Вывод:** `k=0`.

---
## Ошибочные пороги


In [9]:
# тестирование обработка некорректных порогов:
invalid_thresholds = [0, -0.1, 1.1]

for thr in invalid_thresholds:
    try:
        auto_select_k([1,2,3], thr)
    except ValueError as e:
        print(f"threshold={thr} → ValueError: {e}")
    else:
        raise AssertionError(f"Expected ValueError for threshold={thr}")

threshold=0 → ValueError: threshold должен быть в диапазоне (0, 1]
threshold=-0.1 → ValueError: threshold должен быть в диапазоне (0, 1]
threshold=1.1 → ValueError: threshold должен быть в диапазоне (0, 1]


- **`threshold=0` → ValueError**
  Порог равен нижней границе диапазона, но допустимым считается только (0, 1].
  **Вывод:** вызов `ValueError("threshold должен быть в диапазоне (0, 1]")`.

- **`threshold=-0.1` → ValueError**
  Отрицательное значение вне допустимого диапазона.
  **Вывод:** `ValueError("threshold должен быть в диапазоне (0, 1]")`.

- **`threshold=1.1` → ValueError**
  Значение превышает 1, выходит за верхнюю границу диапазона.
  **Вывод:** `ValueError("threshold должен быть в диапазоне (0, 1]")`.

---
## Порог ровно 1.0

In [10]:
# тест с порогом = 1 и большим списком
evs = [0.2, 0.1, 0.05, 0.025]
k_full = auto_select_k(evs, 1.0)
print(f"threshold=1.0 → k={k_full} (должно быть 4)")
assert k_full == len(evs)

threshold=1.0 → k=4 (должно быть 4)


- **`[0.2, 0.1, 0.05, 0.025], threshold=1.0 → k=4`**
  Сумма собственных значений = 0.2 + 0.1 + 0.05 + 0.025 = 0.375.
  - Чтобы охватить 100 % дисперсии, нужно собрать сумму всех компонент:
    \(\frac{0.2 + 0.1 + 0.05 + 0.025}{0.375} = 1.0\).
  **Вывод:** минимальное `k=4`, то есть равное полному числу собственных значений.

---

## Выводы

- Функция `auto_select_k` корректно находит **минимальное** число компонент `k`, обеспечивающее заданный порог объяснённой дисперсии:
  для каждого тестового набора собственных значений были выбраны именно те `k`, при которых накопленная доля ≥ threshold.

- При **пустом списке** собственных значений возвращается `k = 0`, что логично: нет компонент для отбора.

- Для **порога = 1.0** функция возвращает **полное** число компонент, обеспечивая 100 % объяснённой дисперсии.

- **Неправильные пороги** (≤ 0 или > 1) надёжно отбрасываются с выбрасыванием `ValueError("threshold должен быть в диапазоне (0, 1]")`.

---